Created on Mon Apr 20 16:58:00 2020

@author: yifei

2-D line heat source with grondwater flow (uniform grid spacing of 1m)😃

In [ ]:
import numpy as np
import flopy
import os
import matplotlib as mpl
import matplotlib.pyplot as plt

###################################################### MODFLOW Simulation ##############################################################

There are always some issues reading the path of the file, so here are some solutions:
1. Use double backslash \\ to seperate the directories
2. Use normal slash /
3. Add r (raw) in front of the path

In [ ]:
modelname = 'uniform_linesource'
modelpath = r'D:\Google Drive\UIUC grad courses\CEE 599 Master Thesis\Preliminary model MT3D\Flopy Model\Uniform_linesource'
mf = flopy.modflow.Modflow(modelname, exe_name='mf2005', model_ws=modelpath)

# Model spatial and time discretization
Lx = 500.
Ly = 300.
ztop = 1.
zbot = 0.
nlay = 1
nrow = 300
ncol = 500
delr = Lx/ncol
delc = Ly/nrow

DIS package
1. We can specify delr, delc and botm as an 1-D numpy array, to have variable-spcaing mesh
2. In Flopy, the order is (lay,row,col) with 0-based index; while in GMS it is (k,i,j) starting from index 1
3. delr means dx along row. So in the Cartesian coordinate system, this aligns with x-axis

In [ ]:
# create DIS object
dis = flopy.modflow.ModflowDis(mf, nlay, nrow, ncol, delr=delr, delc=delc,top=ztop, botm=zbot,steady = True)

In [ ]:
BAS package

In [ ]:
# create BAS object
ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)
ibound[:, :, 0] = -1
ibound[:, :, -1] = -1 #left and right constant head boundary
strt = 240* np.ones((nlay, nrow, ncol), dtype=np.float32)
strt[:, :, 0] = 250
bas = flopy.modflow.ModflowBas(mf, ibound=ibound, strt=strt)

In [ ]:
LPF package

In [ ]:
# Add LPF package to the MODFLOW model
hk = 10 * np.ones((nlay,nrow,ncol), dtype=np.float32)
lpf = flopy.modflow.ModflowLpf(mf, hk=hk, hani = 1., vka=1., ipakcb=53)

Output control package
1. the stress period data is a dictionary with a tuple as key and various instructions as value
2. The first element of the tuple is the stress period and the second is the timestep
3. For instructions, we can print/save head, drawdown, cell budget..etc
4. The compact mode is needed for MODPATH and creates way smaller output files

In [ ]:
# Add OC package to the MODFLOW model
spd = {(0, 0): ['print head', 'print budget', 'save head', 'save budget']}
oc = flopy.modflow.ModflowOc(mf, stress_period_data=spd, compact=True)

The generated flow-transport link file should be included in the MT3DMS simulation directory

In [ ]:
# PCG package for matrix computation
pcg = flopy.modflow.ModflowPcg(mf)

# Creat LMT package to link to MT3DMS
lmt = flopy.modflow.ModflowLmt(mf, output_file_name='mt3d_link.ftl')

# Write the MODFLOW model input files and check
mf.write_input()
mf.check()

# Run the MODFLOW model
success, buff = mf.run_model()

###################################################### MT3DMS Simulation ##############################################################

In [ ]:
mt_exe_pth = r'D:\Hydro modeling\MT3DMS\bin\mt3dms5b.exe'
mt_modelpth = r'D:\Google Drive\UIUC grad courses\CEE 599 Master Thesis\Preliminary model MT3D\Flopy Model\Uniform_linesource_MT3D'
mt = flopy.mt3d.Mt3dms(modflowmodel=mf, modelname=modelname, exe_name= mt_exe_pth ,ftlfilename='mt3d_link.ftl',model_ws = mt_modelpth)

In [ ]:
# basic transport package
porosity = 0.3 
output_times = [6.0,60.,360.] #Elasped time at which simulation results are saved, # of entries euqal to nprs
species_names = ['Temperature']
btn = flopy.mt3d.Mt3dBtn(mt, prsity=porosity, icbund = 1, sconc=273, ncomp=1, perlen = 360, nper=1, nstp = 60, tsmult = 1.0, 
                         nprs = 3, timprs = output_times, obs = [(0,149,150)], nprobs = 6, cinact = -1, chkmas=True,
                         species_names = species_names)

# advaction package 
MMOC = 2 #The modified MOC is used here 
RK4 = 2
adv = flopy.mt3d.Mt3dAdv(mt, mixelm = MMOC, percel=1, itrack = RK4, nplane = 0, nph = 16, npsink = 16)

# dispersion package
dsp = flopy.mt3d.Mt3dDsp(mt, al=0.00001, trpt=0.1, trpv=0.1, dmcoef=0.065)

Source-Sink Mxing Package
1. The source-sink mixing data is also a dictionary, with the key value corresponding to the stress period.
2. The value is a list of tuples with each encompassing node location, bc and type of bc
3. The maximum ssm defined should not exceed mxss

In [ ]:
# source/sink mixing package
ssm_data = {}
itype = flopy.mt3d.Mt3dSsm.itype_dict()
ssm_data[0] = [(0, 149, 149, 1.23, itype['MAS'])]
for k in range(nrow):
    ssm_data[0].append((0, k, 0, 273.0, itype['CHD']))
for k in range(nrow):
    ssm_data[0].append((0, k, 499, 273.0, itype['CHD']))    
ssm = flopy.mt3d.Mt3dSsm(mt, mxss=1002, stress_period_data=ssm_data)

In [ ]:
#chemical reaction package
rct = flopy.mt3d.Mt3dRct(mt, isothm=1, ireact=0, igetsc=0, rhob=1855.0, sp1=2.02381e-4 )

# matrix solver package
gcg = flopy.mt3d.Mt3dGcg(mt, cclose=1e-4)

# write mt3dms input
mt.write_input()

# run mt3dms
mt.run_model()

Flopy Plotting Utility
1. Head information in .hds, cell Budget in .cbc, concentration in .CNF. (all binary files)
2. The plot_arrray() is a wrapper for matplotlib im.show(), thus return a matplotlib quadmesh object
3. In previous version of Flopy, someone used ModelMap, but it has been took place by PlotMapView

In [ ]:
#plot MODFLOW head field
fname = os.path.join(modelpath, 'uniform_linesource.hds' )
hdobj = flopy.utils.HeadFile(fname)
head = hdobj.get_data()

fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
mapview = flopy.plot.PlotMapView(model=mf)
quadmesh = mapview.plot_array(head, masked_values=[999.], alpha=0.5,cmap="viridis")
cb = plt.colorbar(quadmesh, shrink=0.75)

#These creates the contour head plot
#levels = np.arange(head.min(), head.max(), 1)
#contour_set = mapview.contour_array(head, masked_values=[999.], levels=levels)
#manual_locations = [(0,150),(50,150),(100,150),(150,150),(200,150),(250,150),(300,150),(350,150),(400,150),(450,150),(500,150)]
#ax.clabel(contour_set,inline = 1,fontsize = 10,manual=manual_locations)

ax.set_title('head')
ax.set_xlim([-0.1,500.1])
ax.set_xlabel('x')
ax.set_ylabel('y')

In [ ]:
#Plot concentration along a certain axis

ucnobj_fp = bf.UcnFile(r'D:\Google Drive\UIUC grad courses\CEE 599 Master Thesis\Preliminary model MT3D\Flopy Model\uniform linesource_MT3D\MT3D001.UCN')
times_f = ucnobj_fp.get_times() #this get an list of the output times as defined in BTN
times_f1 = times_f[0] #6 days
times_f2 = times_f[1] #60 days
times_f3 = times_f[2] #360 days

conc1_fp = ucnobj_fp.get_data(totim=times_f1) #get the concentration at all nodes at time1
conc2_fp = ucnobj_fp.get_data(totim=times_f2)
conc3_fp = ucnobj_fp.get_data(totim=times_f3)

plt.plot(x,conc3_fp,'r--',alpha = 0.5,label = 'Flopy')
plt.title('t = 360 days')
plt.legend()

#Plot concentraion map 
mf = flopy.modflow.Modflow.load(r'D:\Google Drive\UIUC grad courses\CEE 599 Master Thesis\Preliminary model MT3D\Flopy Model\uniform linesource\uniform_linesource.nam')
fig = plt.figure(figsize=(15, 10)) #MODFLOW Nam file need to be loaded to get the desired information

ax = fig.add_subplot(1, 1, 1, aspect='equal')
mapview = flopy.plot.PlotMapView(model=mf) #The read MODFLOW model is needed here
quadmesh = mapview.plot_array(conc3_fp, masked_values=[999.], alpha=0.5,cmap = 'jet')
cb = plt.colorbar(quadmesh, shrink=0.75)
ax.set_title('conc')
ax.set_xlim([0,500])
ax.set_xlabel('x')
ax.set_ylabel('y')